<a href="https://colab.research.google.com/github/8persy/algoritms_colab/blob/main/%22%D0%9F%D0%BE%D0%B8%D1%81%D0%BA_%D0%BA%D1%80%D0%B0%D1%82%D1%87%D0%B0%D0%B9%D1%88%D0%B5%D0%B3%D0%BE_%D0%BF%D1%83%D1%82%D0%B8_11_311%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поиск кратчайшего пути в графе

**Поиск кратчайшего пути в графе** - это нахождение самого короткого пути между двумя вершинами в графе

## Переборный алгоритм

Сложность - O(n!)

1. Перебираем все возможные перестановки вершин графа для нахождения путей от начальной вершины до конечной.

2. Для каждой перестановки вычисляем общее расстояние, проходя по ребрам между вершинами. Если путь до текущей вершины не существует, то расстояние устанавливаем как бесконечность.

3. Для каждой перестановки, которая начинается с начальной вершины и заканчивается на конечной, вычисляем общее расстояние пути.

4. В конце выбираем путь с наименьшим общим расстоянием, который и будет являться кратчайшим путем от начальной вершины до конечной.

### Чекпоинт №1

In [47]:
from itertools import permutations

def find_shortest_path(graph, start, end):
    path = 'A'*1000
    distance = 1000000
    for perm in permutations(graph):
      curr_path = perm[0]
      curr_distance = 0
      for i in range(len(perm)):
        if perm[i] == end:
          break
        if perm[i+1] in graph[perm[i]]:
          curr_path = curr_path + perm[i+1]
          curr_distance += graph[perm[i]][perm[i+1]]
        else:
          curr_distance += 1000000000

      if (curr_distance < distance or (curr_distance == distance and len(curr_path) < len(path))) and start in curr_path and end in curr_path:
        distance = curr_distance
        path = curr_path
    return path, distance


In [49]:
graph = {
    'A': {'B': 2, 'C': 5},
    'B': {'C': 1, 'D': 3},
    'C': {'D': 2},
    'D': {}
}

In [50]:
start_node = 'A'
end_node = 'D'

In [51]:
shortest_path, distance = find_shortest_path(graph, start_node, end_node)

print(f'Кратчайший путь от вершины {start_node} к вершине {end_node}: {shortest_path}')
print(f'Длина кратчайшего пути: {distance}')

Кратчайший путь от вершины A к вершине D: ABD
Длина кратчайшего пути: 5


## Алгоритм Дейкстры

Сложность - O((V + E) * log(V))

Алгоритм Дейкстры основывается на принципе жадного подхода, который заключается в выборе на каждом шаге наилучшего текущего решения без учета возможных последствий этого выбора.

! Алгоритм Дейкстры требует, чтобы все веса ребер графа были неотрицательными

1. Начинаем с выбора стартовой вершины и установки расстояния от нее до нее самой равным нулю, а до всех остальных - бесконечности.
2. Затем выбираем вершину с минимальным известным расстоянием и рассматриваем все ребра, исходящие из нее.
3. Если общее расстояние от стартовой вершины до текущей вершины меньше, чем известное до этого, то это значение обновляется.
4. Этот процесс повторяется для всех вершин, пока все вершины не будут посещены.

### Чекпоинт №2

In [53]:
def dijkstra(graph, start):
    res = {start:0}
    for i in graph:
      if i != start:
        res[i] = 1000
    visited = set()

    while len(visited) < len(graph):
      unvisited_vertices = [vertex for vertex in graph if vertex not in visited]
      curr_ver = min(unvisited_vertices, key = lambda x : res[x])

      visited.add(curr_ver)

      for ver, weight in graph[curr_ver].items():
        if res[curr_ver] + weight < res[ver]:
          res[ver] = res[curr_ver] + weight

    return res

In [54]:
graph = {
    'A': {'B': 2, 'C': 4},
    'B': {'A': 2, 'C': 1, 'D': 7},
    'C': {'A': 4, 'B': 1, 'D': 3},
    'D': {'B': 7, 'C': 3}
}

In [55]:
start_vertex = 'A'

In [56]:
shortest_distances = dijkstra(graph, start_vertex)

print(f"Кратчайшие расстояния от вершины {start_vertex}:")
for vertex, distance in shortest_distances.items():
    print(f"До вершины {vertex}: {distance}")

Кратчайшие расстояния от вершины A:
До вершины A: 0
До вершины B: 2
До вершины C: 3
До вершины D: 6


## Алгоритм Флойда-Уоршелла

Этот алгоритм основан на динамическом программировании и имеет сложность O(n^3), где n - количество вершин в графе.

1. Инициализируем матрицу расстояний, где в ячейке (i, j) содержится вес ребра между вершинами i и j, либо бесконечность, если ребра нет.
2. Последовательно проходим по всем вершинам и для каждой вершины параллельно находим кратчайшие пути между всеми парами вершин с учетом этой вершины.
3. Обновляем матрицу расстояний, добавляя промежуточные вершины на путях, если они уменьшают суммарный вес пути.
4. После завершения алгоритма матрица расстояний содержит длины кратчайших путей между всеми парами вершин.

### Чекпоинт №3

In [67]:
def floyd_warshall(graph):
    vertices = list(graph.keys())
    n = len(vertices)
    res = [[10000] * n for _ in range(n)]

    for i in range(n):
        res[i][i] = 0
        ver = vertices[i]
        for neighbor, weight in graph[ver].items():
            j = vertices.index(neighbor)
            res[i][j] = weight

    for k in range(n):
        for i in range(n):
            for j in range(n):
                res[i][j] = min(res[i][j], res[i][k] + res[k][j])

    return res

In [68]:
graph = {
    'A': {'B': 2, 'C': 4},
    'B': {'A': 2, 'C': 1, 'D': 7},
    'C': {'A': 4, 'B': 1, 'D': 3},
    'D': {'B': 7, 'C': 3}
}

In [69]:
result = floyd_warshall(graph)
for row in result:
    print(row)

[0, 2, 3, 6]
[2, 0, 1, 4]
[3, 1, 0, 3]
[6, 4, 3, 0]
